In [5]:
import torch
from torch import nn # This imports the neural network library (All the features we need are here)

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset, Dataset
import torch.nn.functional as F

import monai
from monai.losses import DiceLoss

from torchsummary import summary
from tqdm import tqdm

import matplotlib as plt
import matplotlib.pyplot as plt
import numpy as np

print(torch.__version__)

2.4.1+cu121


In [6]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

transform = A.Compose([
    A.Resize(512, 512),
    ToTensorV2()
], p=1.0)

In [ ]:
class VOCDataset(torchvision.datasets.VOCSegmentation):
    def __getitem__(self, idx):
        img, mask = super().__getitem__(idx)

        # Convert to numpy arrays
        image = np.array(img)
        mask = np.array(mask)

        # Apply transformations
        transformed = transform(image=image, mask=mask)
        image = transformed['image']
        mask = transformed['mask']

        # Normalize image
        image = image.float() / 255.0

        # Add channel dimension to mask [H, W] -> [1, H, W]
        # No need for torch.from_numpy since mask is already a tensor
        mask = mask.unsqueeze(0)

        return image, mask.long()

# Create datasets
train_dataset = VOCDataset(root='data', year='2012', image_set='train', download=True)
val_dataset = VOCDataset(root='data', year='2012', image_set='trainval', download=True)

# Test the shapes
image, mask = train_dataset[0]
print("Single sample shapes:")
print(f"Image shape: {image.shape}")    # Should be [3, 512, 512]
print(f"Mask shape: {mask.shape}")      # Should be [1, 512, 512]
print(f"Unique mask values: {torch.unique(mask)}")

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Test batch shapes
batch_images, batch_masks = next(iter(train_dataloader))
print("\nBatch shapes:")
print(f"Batch images shape: {batch_images.shape}")  # Should be [4, 3, 512, 512]
print(f"Batch masks shape: {batch_masks.shape}")    # Should be [4, 1, 512, 512]

Using downloaded and verified file: data/VOCtrainval_11-May-2012.tar
Extracting data/VOCtrainval_11-May-2012.tar to data
